# Overview
In this lab we will call **DALL-E** model API from Lang chain application for getting social media pimage illustrationost by sending social media post as an input.

**DALL-E** is a powerful AI model that can generate and edit images based on natural language prompts. **Image client** is a Python library that allows you to interact with DALL-E and other image models using a simple and intuitive interface. GPT-4 is a state-of-the-art language model that can understand and generate natural language or code. By combining these three components, you can create amazing applications that use natural language to generate and manipulate images. For example, you can use GPT-4 to generate a prompt for DALL-E, such as “a cat wearing a hat”, and then use image client to send the prompt to DALL-E and receive an image as a response. You can also use image client to edit the image, such as changing the color or shape of the hat, and then use GPT-4 to generate a description of the edited image. 

### Steps:

1. Generate a prompt for DALL-E using GPT model
2. Use image client to send the prompt to DALL-E and receive an image as a response.

### Import python libraries

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

### Get social media post from previous Lab

In [ ]:
# Open the file
with open('social_media_post.txt', 'r') as f:
    social_media_post = f.read()

# Print the content
print(social_media_post)

### Get Environment variables
The provided code is importing the `load_dotenv` function from the `dotenv` module and using it to load environment variables from a `.env`

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

In [ ]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

dalle_model = os.getenv("AZURE_OPENAI_DALLE_MODEL")
print(dalle_model)

### Use GPT model to generate prompt for DALL-E model
This Python code snippet is used to generate a DALL-E prompt using the Azure OpenAI Service with the GPT-4 model. The DALL-E prompt is generated based on the social media post created in the previous steps.

- A `system_template` string is defined that describes the task of the AI model.
- A `user_prompt` is created using the `PromptTemplate` class. This prompt includes a template for the DALL-E prompt and a placeholder for the social media post.
- The `HumanMessagePromptTemplate` class is used to create a human message prompt from the `user_prompt`.
- An instance of `AzureChatOpenAI` is created, which is a wrapper for the Azure OpenAI Service. The API version and the model name are passed as arguments.
- The `AzureChatOpenAI` instance is called with the `dalle_messages` as an argument. This sends a request to the Azure OpenAI Service to generate a DALL-E prompt based on the `dalle_messages`.
- An instance of `LLMChain` is created with the `AzureChatOpenAI` instance, the chat prompt, and the output key as arguments. This object represents a chain of language model calls.
- The content of the response from the Azure OpenAI Service is stored in the `dalle_prompt` variable.

In [ ]:
# Use GPT-4 to generate a DALL-E prompt
system_template="You are a helpful large language model that generates DALL-E prompts, that when given to the DALL-E model can generate beautiful high-quality images to use in social media posts about a podcast on technology.  Good DALL-E prompts will contain mention of related objects, and will not contain people or words.  Good DALL-E prompts should include a reference to podcasting along with items from the domain of the podcast guest.\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

user_prompt=PromptTemplate(
    template="Create a DALL-E prompt to create an image to post along with this social media text: {social_media_copy}",
    input_variables=["social_media_copy"],
)
human_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Get formatted messages for the chat completion
dalle_messages = chat_prompt.format_prompt(social_media_copy={social_media_post}).to_messages()

# Call Azure OpenAI Service to get a DALL-E prompt 
print("Calling GPT-4 model on Azure OpenAI Service to get a DALL-E prompt...\n")
gpt4 = AzureChatOpenAI(
    openai_api_version=api_version,
    deployment_name=model,
    openai_api_type = "azure",
)

output = gpt4(dalle_messages)
dalle_prompt = output.content

dalle_prompt_chain = LLMChain(llm=gpt4, prompt=chat_prompt, output_key="dalle_prompt")

print("DALL-E Prompt:")
print(dalle_prompt)

### Save DALL-E prompt to file

In [ ]:
# Specify the file path
file_path = "dalle_prompt.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(dalle_prompt)

### Alter DALL-E prompt
Append "high-quality digital art" to the generated DALL-E prompt as a guidance for the model how to create an image

In [ ]:
# Append "high-quality digital art" to the generated DALL-E prompt
dalle_prompt = dalle_prompt + ", high-quality digital art"
print("dalleprompt: ",dalle_prompt)

### Make a call to DALL-E model on the Azure OpenAI Service to generate an image 
This code is creating AzureOpenAI client and makes actual call to **DALL-E** model to create an image

*You can experiment further and try your own `dalle_prompt`*

In [ ]:
from openai import AzureOpenAI
import requests
from PIL import Image
import json

client = AzureOpenAI(
    api_version="2024-02-01",  
    api_key=api_key,  
    azure_endpoint=azure_endpoint
)

result = client.images.generate(
    model=dalle_model, # the name of your DALL-E 3 deployment
    prompt=dalle_prompt,
    n=1
)

json_response = json.loads(result.model_dump_json())

## Get the image and display it

In [ ]:
# Initialize the image path (note the filetype should be png)
image_path = 'postImage.png'

# Retrieve the generated image
image_url = json_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(image_url).content  # download the image
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

# Display the image in the default image viewer
image = Image.open(image_path)
image